Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.702874
Minibatch accuracy: 25.0%
Validation accuracy: 10.2%
Minibatch loss at step 50: 1.445576
Minibatch accuracy: 56.2%
Validation accuracy: 37.4%
Minibatch loss at step 100: 0.849272
Minibatch accuracy: 68.8%
Validation accuracy: 69.8%
Minibatch loss at step 150: 0.537236
Minibatch accuracy: 87.5%
Validation accuracy: 75.9%
Minibatch loss at step 200: 0.669470
Minibatch accuracy: 75.0%
Validation accuracy: 77.5%
Minibatch loss at step 250: 0.416131
Minibatch accuracy: 87.5%
Validation accuracy: 77.2%
Minibatch loss at step 300: 1.131178
Minibatch accuracy: 75.0%
Validation accuracy: 77.9%
Minibatch loss at step 350: 0.854559
Minibatch accuracy: 87.5%
Validation accuracy: 77.6%
Minibatch loss at step 400: 0.506941
Minibatch accuracy: 81.2%
Validation accuracy: 80.5%
Minibatch loss at step 450: 0.856375
Minibatch accuracy: 81.2%
Validation accuracy: 80.7%
Minibatch loss at step 500: 0.762644
Minibatch accuracy: 75.0%
Validation accuracy: 80.8%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---
Solution : 

Added Pool_1 and Pool_2 layers. 

In [16]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv_1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    relu_1 = tf.nn.relu(conv_1 + layer1_biases)
    
    #pool layer after conv 1
    pool_1 = tf.nn.max_pool(relu_1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    print('\nPool layer 1 size :')
    print(pool_1)
    
    conv_2 = tf.nn.conv2d(pool_1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    relu_2 = tf.nn.relu(conv_2 + layer2_biases)
    
    #pool layer after conv 2
    pool_2 = tf.nn.max_pool(relu_2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    print('\nPool layer 2 size :')
    print(pool_2)
    
    shape = pool_2.get_shape().as_list()
    reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))


Pool layer 1 size :
Tensor("MaxPool:0", shape=(16, 14, 14, 16), dtype=float32)

Pool layer 2 size :
Tensor("MaxPool_1:0", shape=(16, 7, 7, 16), dtype=float32)

Pool layer 1 size :
Tensor("MaxPool_2:0", shape=(10000, 14, 14, 16), dtype=float32)

Pool layer 2 size :
Tensor("MaxPool_3:0", shape=(10000, 7, 7, 16), dtype=float32)

Pool layer 1 size :
Tensor("MaxPool_4:0", shape=(10000, 14, 14, 16), dtype=float32)

Pool layer 2 size :
Tensor("MaxPool_5:0", shape=(10000, 7, 7, 16), dtype=float32)


Pool layer 1 shrinks the feature map in size

(16, 28, 28, 16) ----> (16, 14, 14, 16)

Pool layer 2 shrinks the feature map in size

(16, 14, 14, 16) ----> (16, 7, 7, 16)


In [17]:
num_steps = 8001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.384141
Minibatch accuracy: 0.0%
Validation accuracy: 9.8%
Minibatch loss at step 500: 0.849342
Minibatch accuracy: 87.5%
Validation accuracy: 81.6%
Minibatch loss at step 1000: 0.231783
Minibatch accuracy: 93.8%
Validation accuracy: 84.4%
Minibatch loss at step 1500: 1.090719
Minibatch accuracy: 62.5%
Validation accuracy: 85.8%
Minibatch loss at step 2000: 0.232114
Minibatch accuracy: 93.8%
Validation accuracy: 86.5%
Minibatch loss at step 2500: 0.299323
Minibatch accuracy: 93.8%
Validation accuracy: 86.4%
Minibatch loss at step 3000: 0.521670
Minibatch accuracy: 87.5%
Validation accuracy: 87.5%
Minibatch loss at step 3500: 0.325275
Minibatch accuracy: 87.5%
Validation accuracy: 87.2%
Minibatch loss at step 4000: 0.059035
Minibatch accuracy: 100.0%
Validation accuracy: 87.4%
Minibatch loss at step 4500: 0.168560
Minibatch accuracy: 93.8%
Validation accuracy: 87.9%
Minibatch loss at step 5000: 0.447678
Minibatch accuracy: 87.5%
Validation accuracy

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

Solution:
Architecture
Convolution layer 1
Relu layer 1
Pool Layer 1
Convolution layer 2
Relu layer 2
Pool Layer 2
Convolution layer 3
Relu layer 3
Fully connected layers


In [42]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
regularization_parameter = 0.002

graph = tf.Graph()

with graph.as_default():

  # Input data 
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.

  #- size : (5, 5, 1, 16)
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
    
  #- size : (5, 5, 16, 16)
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

  #- size : (5, 5, 16, 16)
  layer3_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

  #- size :(784, 64)
  layer4_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  #- size :(64, 10)
  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

  
  # Model.
  def model(data):
    # conv_1 result size (16, 28, 28, 16)
    conv_1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    relu_1 = tf.nn.relu(conv_1 + layer1_biases)
    #dropOut_1 = tf.nn.dropout(relu_1,0.45)
    
    #pool layer after conv 1
    pool_1 = tf.nn.max_pool(relu_1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    # conv_2 result size (16, 14, 14, 16)
    conv_2 = tf.nn.conv2d(pool_1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    relu_2 = tf.nn.relu(conv_2 + layer2_biases)
    #dropOut_2 = tf.nn.dropout(relu_2,0.45)
    
    #pool layer after conv 2
    pool_2 = tf.nn.max_pool(relu_2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')

    # conv_2 result size (16, 7, 7, 16)
    conv_3 = tf.nn.conv2d(pool_2, layer3_weights, [1, 1, 1, 1], padding='SAME')
    relu_3 = tf.nn.relu(conv_3 + layer3_biases)
    #dropOut_2 = tf.nn.dropout(relu_3,0.2)
    
    shape = relu_3.get_shape().as_list()
    reshape = tf.reshape(relu_3, [shape[0], shape[1] * shape[2] * shape[3]])
    
    #hidden layer result size (16, 64)
    hidden = tf.nn.relu(tf.matmul(reshape, layer4_weights) + layer4_biases)

    return tf.matmul(hidden, layer5_weights) + layer5_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + regularization_parameter * (tf.nn.l2_loss(layer5_weights)
    + tf.nn.l2_loss(layer4_weights) + tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer2_weights) + tf.nn.l2_loss(layer1_weights))
    
  global_step = tf.Variable(0) 
  learning_rate = tf.train.exponential_decay(0.055, global_step, 1000, 0.80, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  #Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [43]:
num_steps = 12001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.888076
Minibatch accuracy: 0.0%
Validation accuracy: 10.1%
Minibatch loss at step 500: 1.071677
Minibatch accuracy: 81.2%
Validation accuracy: 82.7%
Minibatch loss at step 1000: 0.581544
Minibatch accuracy: 93.8%
Validation accuracy: 85.0%
Minibatch loss at step 1500: 1.254761
Minibatch accuracy: 68.8%
Validation accuracy: 86.6%
Minibatch loss at step 2000: 0.657728
Minibatch accuracy: 93.8%
Validation accuracy: 86.7%
Minibatch loss at step 2500: 0.647746
Minibatch accuracy: 93.8%
Validation accuracy: 87.7%
Minibatch loss at step 3000: 0.704708
Minibatch accuracy: 87.5%
Validation accuracy: 87.7%
Minibatch loss at step 3500: 0.610302
Minibatch accuracy: 87.5%
Validation accuracy: 87.5%
Minibatch loss at step 4000: 0.351583
Minibatch accuracy: 100.0%
Validation accuracy: 88.2%
Minibatch loss at step 4500: 0.691286
Minibatch accuracy: 87.5%
Validation accuracy: 88.8%
Minibatch loss at step 5000: 0.685905
Minibatch accuracy: 87.5%
Validation accurac

Accuracy has little increased but best for this dataset of all different models. Regularization parameter, learning rate, num_steps are tuned for better accuracy.